In [ ]:
#!rendercv new "Mason Youngblood"

In [4]:
!rendercv render "Mason_Youngblood_CV.yaml"

                                                                         
Welcome to RenderCV! Some useful links:                                  
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Title             ┃                                              Link ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ RenderCV App      │                              https://rendercv.com │
│ Documentation     │                         https://docs.rendercv.com │
│ Source code       │             https://github.com/rendercv/rendercv/ │
│ Bug reports       │      https://github.com/rendercv/rendercv/issues/ │
│ Feature requests  │      https://github.com/rendercv/rendercv/issues/ │
│ Discussions       │ https://github.com/rendercv/rendercv/discussions/ │
│ RenderCV Pipeline │    https://github.com/rendercv/rendercv-pipeline/ │
└───────────────────┴───────────────────────────────────────────────────┘
╭─────────────────────────────────────

In [1]:
import re
import yaml

def parse_bibtex(input_text):
    blocks = []
    cur = []
    for line in input_text.splitlines():
        if line.strip().startswith("#"):
            if cur:
                blocks.append('\n'.join(cur))
                cur = []
            blocks.append(line.strip())
        elif line.strip().startswith("@"):
            if cur:
                blocks.append('\n'.join(cur))
                cur = []
            cur.append(line.strip())
        else:
            cur.append(line.rstrip())
    if cur:
        blocks.append('\n'.join(cur))
    return blocks

def parse_fields(entry):
    fields = {}
    for match in re.finditer(r'(\w+)\s*=\s*\{((?:[^{}]|{[^{}]*})*)\}', entry, re.DOTALL):
        key, value = match.group(1).lower(), match.group(2).strip()
        fields[key] = value
    return fields

def apa_author_list(authors_raw, your_last="Youngblood", your_first="Mason"):
    authors = [a.strip() for a in re.split(r'\s+and\s+', authors_raw) if a.strip()]
    apa_names = []
    your_apa = f"**{your_last}, M.**"
    found_your_name = False
    your_index = None

    for i, a in enumerate(authors):
        m = re.match(r'([\w\-]+),\s*([\w\.\- ]+)', a)
        if m:
            last, firsts = m.groups()
            initials = "".join([f"{name.strip()[0]}." for name in firsts.strip().split() if name.strip()])
            apa = f"{last}, {initials}"
            if last.lower() == your_last.lower():
                apa_names.append(your_apa)
                found_your_name = True
                your_index = i
            else:
                apa_names.append(apa)
        else:
            if your_last.lower() in a.lower():
                apa_names.append(your_apa)
                found_your_name = True
                your_index = i
            else:
                apa_names.append(a)

    # Remove duplicates of your name
    apa_names = [a for i, a in enumerate(apa_names) if a != your_apa or i == apa_names.index(your_apa)]
    n = len(apa_names)

    # If not found, add your name at the end
    if not found_your_name:
        apa_names.append(your_apa)
        your_index = len(apa_names) - 1
        n += 1

    # APA style
    if n == 1:
        return apa_names[0]
    elif n == 2:
        return f"{apa_names[0]} and {apa_names[1]}"
    elif n <= 5:
        return ", ".join(apa_names[:-1]) + ", and " + apa_names[-1]
    else:
        # >5 authors, et al. logic, no commas, no "and" unless your name is last
        if apa_names[0] == your_apa:
            return f"{your_apa} et al..."
        elif apa_names[-1] == your_apa:
            return f"{apa_names[0]} et al... and {your_apa}"
        else:
            return f"{apa_names[0]} et al... {your_apa}..."

def bibtex_to_yaml(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as f:
        text = f.read()
    blocks = parse_bibtex(text)
    yaml_entries = []
    idx = 1
    for block in blocks:
        if block.startswith("#"):
            section = block.lstrip("#").strip()
            yaml_entries.append({
                'title': section,
                'journal': "",
                'date': "",
                'authors': [""],  # Empty authors
                'new': "",  # Empty authors_new for section headers
                'index': ""
            })
        elif block.startswith("@"):
            fields = parse_fields(block)
            title = fields.get('title', '').replace('\n', ' ').replace('{', '').replace('}', '').strip()
            journal = fields.get('journal', '').replace('\n', ' ').strip()
            doi = fields.get('doi', '').replace('\n', ' ').strip()
            url = fields.get('url', '').replace('\n', ' ').strip()
            year = fields.get('year', '').replace('\n', ' ').strip()
            authors_raw = fields.get('author', '')
            authors_apa = apa_author_list(authors_raw)
            yaml_entries.append({
                'index': idx,
                'title': title,
                'authors': [""],  # Empty authors
                'new': authors_apa,  # APA formatted authors in new field
                'journal': journal,
                'doi': doi,
                'url': url,
                'date': year
            })
            idx += 1
    with open(output_file, "w", encoding="utf-8") as f:
        yaml.dump(yaml_entries, f, sort_keys=False, allow_unicode=True, width=1000)

# Example usage:
bibtex_to_yaml("citations.bib", "citations.yaml")